In [10]:
using Rx
using BenchmarkTools

In [11]:
function withRegularArray()
    values = [ i for i in 1:1000 ]
    keep = Array{Int, 1}()
    for value in values
         push!(keep, value ^ 2)
    end
    return keep
end

withRegularArray (generic function with 1 method)

In [12]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [13]:
@CreateMapOperator(Squared, (d) -> d ^ 2)

function withPregeneratedOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> SquaredMapOperator{Int, Int}(), keep)
    return keep.values
end

withPregeneratedOperator (generic function with 1 method)

In [14]:
function withBaseMapOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> map(Int, Int, (d) -> d ^ 2), keep)
    return keep.values
end

withBaseMapOperator (generic function with 1 method)

In [18]:
@btime withRegularArray();

  6.908 μs (11 allocations: 24.33 KiB)


In [19]:
@btime withPregeneratedOperator();

  7.244 μs (15 allocations: 24.41 KiB)


In [20]:
@btime withBaseMapOperator();

  80.367 μs (2483 allocations: 62.98 KiB)
